# Python Type Annotations, Dataclasses, and Serialization with Datafiles

In [ ]:
import sys

assert sys.version_info > (3, 7)

sys.version_info

In [ ]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

In [ ]:
try:
    item = InventoryItem()
except TypeError as e:
    print(repr(e))

In [ ]:
item = InventoryItem("Widget", 1.99)

item

In [ ]:
item.name

In [ ]:
item.unit_price = 2.99

item

# Type Annotations

In [ ]:
assert sys.version_info > (3, 6)

### Argument Annotations

In [ ]:
def greet(name: str):
    print("Hello, " + name)
    
greet("Jace")

In [ ]:
greet

In [ ]:
greet.__annotations__

In [ ]:
import typing  # stdlib

hints = typing.get_type_hints(greet)

hints  # `dict` with real classes

In [ ]:
import inspect  # stdlib

signature = inspect.signature(greet)  

signature  # `Signature` object

In [ ]:
signature.parameters

In [ ]:
signature.parameters['name']

In [ ]:
# POSITIONAL_OR_KEYWORD: greet(name)
# KEYWORD_ONLY: greet(*, name)
# VAR_POSITIONAL: greet(*names)
# VAR_KEYWORD: greet(**names)

signature.parameters['name'].kind

In [ ]:
signature.parameters['name'].annotation

### Return Annotations

In [ ]:
from decimal import Decimal

def add_tax(subtotal, rate=0.06) -> Decimal:
    cents = Decimal('0.01')
    return Decimal(subtotal * (1 + rate)).quantize(cents)

add_tax(4.99)

In [ ]:
inspect.signature(add_tax).return_annotation

### Variable Annotations

In [ ]:
class Person:
    name: str

In [ ]:
Person.__annotations__

### Optional Values

In [ ]:
from typing import Optional

def fill(password: Optional[str]):
    if password is not None:
        ...
        
fill("abc123")
fill(None)

### Homogeneous Lists 

In [ ]:
from typing import List

def print_one_more_than(numbers: List[int]):
    for number in numbers:
        print(number + 1)

### Mixed Types

In [ ]:
from typing import Union

def print_items_or_keys(values: Union[list, dict]):
    for value in values:
        print(value)

### ⚠️ Circular Annotations

In [ ]:
class Node:
    
    def connect_edge(edge: 'Edge'):
        pass
    

class Edge:
    def connect_node(node: Node):
        pass

In [ ]:
from __future__ import annotations  


class Node:
    
    def connect_edge(edge: Edge):
        pass
     

class Edge:
    def connect_node(node: Node):
        pass

# Type Checking (with mypy)

In [ ]:
# pip install mypy==0.720

In [ ]:
from mypy import api

def mypy(filename):
    """Emulate `$ mypy <filename>` for notebooks."""
    message, _, _ = api.run([filename])
    print(message or "(no errors)")

In [ ]:
%%writefile greet.py

def greet(name: str):
    print("Hello, " + name)
    
greet("Jace")

In [ ]:
mypy('greet.py')

In [ ]:
%%writefile greet2.py

def greet(name: str):
    print("Hello, " + name)
    
greet(42)

In [ ]:
mypy('greet2.py')

In [ ]:
%%writefile people.py

from typing import Iterable, List


class Person:
    
    def __init__(self, name):
        self.name = name

        
def get_people(*names: Iterable[str]) -> List[Person] :
    return [Person(name) for name in names]
    
    
people = get_people("Alice", "Bob")

people[1].age

In [ ]:
mypy('people.py')

# Dataclasses

In [ ]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

### `__init__`

In [ ]:
InventoryItem("Widget A", 1.99)

In [ ]:
InventoryItem("Widge B", 1.99, 300)

In [ ]:
InventoryItem("Widget C", 1.99, quantity_on_hand=400)

In [ ]:
InventoryItem(name="Widget D", unit_price=1.99, quantity_on_hand=500)

In [ ]:
try:
    InventoryItem(name="Widget E")
except TypeError as e:
    print(repr(e))

### `__repr__`

In [ ]:
item = InventoryItem("Widget", 1.99)

repr(item)

In [ ]:
eval("InventoryItem(name='Widget', unit_price=1.99, quantity_on_hand=0)")

### `__eq__`

In [ ]:
item_a = InventoryItem("Widget A", 1.99)
item_b = InventoryItem("Widget B", 1.99)
item_x = InventoryItem("Widget A", 1.99, quantity_on_hand=0)

In [ ]:
item_a == item_b

In [ ]:
item_a == item_x

### Ordered Dataclasses

In [ ]:
@dataclass(order=True)
class Person:
    last_name: str
    first_name: str
        
    def __str__(self):
        return f'{self.first_name} {self.last_name}'

In [ ]:
people = [
    Person(first_name="Alice", last_name="Smith"),
    Person(first_name="Bob", last_name="Smith"),
    Person(first_name="Carl", last_name="Davidson"),
]

for person in people:
    print(person)

In [ ]:
people.sort()

In [ ]:
for person in people:
    print(person)

### Frozen Dataclasses

In [ ]:
@dataclass(frozen=True)
class Badge:
    number: int
        
badges = [Badge(1001), Badge(1002), Badge(1003)]

In [ ]:
try:
    badges[1].number = 1004
except AttributeError as e:
    print(repr(e))

### Field Customization 

In [ ]:
from dataclasses import field

@dataclass(order=True)
class Person:
    name: str = field(compare=False)
    age: int
    
    def __str__(self):
        return f'{self.name} ({self.age})'

In [ ]:
people = [
    Person("Alice Smith", 30),
    Person("Bob Smith", 25),
    Person("Carl Davidson", 41),
]

for person in people:
    print(person)

In [ ]:
people.sort()

In [ ]:
for person in people:
    print(person)

### ⚠️ Custom `__init__`

In [ ]:
@dataclass
class Bill:
    subtotal: float
    tip: float = 0.0
    
    def __post_init__(self):
        self.total = self.subtotal + self.tip

In [ ]:
bill = Bill(12.99, tip=3.00)

bill

In [ ]:
bill.total

### ⚠️ Mutable Default Values

In [ ]:
from typing import List

try:
    
    @dataclass
    class Group:
        members: List[Person] = []
            
except ValueError as e:
    print(repr(e))

In [ ]:
from dataclasses import field

@dataclass
class Group:
    members: List[Person] = field(default_factory=list)

In [ ]:
group = Group()

group.members.append(people[0])

group

### Utilities

In [ ]:
import dataclasses

dataclasses.is_dataclass(item)

In [ ]:
for field in dataclasses.fields(item):
    print(field, end='\n\n')

In [ ]:
dataclasses.asdict(item)

In [ ]:
dataclasses.astuple(item)

# Serialization (with datafiles)

In [ ]:
# pip install datafiles==0.4

In [ ]:
%%sh

rm -rf items

In [ ]:
from datafiles import datafile

@datafile('items/{self.name}.yml')
class MyInventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

In [ ]:
item = MyInventoryItem("widget", 1.99)

In [ ]:
%%sh

cat items/widget.yml

In [ ]:
item.quantity_on_hand += 100

In [ ]:
%%sh

cat items/widget.yml

In [ ]:
%%writefile items/widget.yml

unit_price: 2.5  # was 3.0
quantity_on_hand: 100

In [ ]:
item.unit_price

In [ ]:
from datafiles import Missing

item = MyInventoryItem("widget", Missing)

assert item.unit_price == 2.5
assert item.quantity_on_hand == 100

item